In [28]:
import requests as requests
import pandas as pd
from bs4 import BeautifulSoup

response = requests.get('https://minecraft.wiki/smelting')
print(response.status_code)
soup = BeautifulSoup(response.content, 'html.parser')

print(soup.p.get_text())

200
Smelting is the process of obtaining refined goods from raw materials by heating them in a furnace, blast furnace or smoker. When items are smelted in either type of furnace, experience is dropped. Like crafting, smelting uses recipes to determine what item is produced.



In [33]:
dfl = pd.read_html(response.content, header=0)
df = dfl[0]
dfstr = df.astype(str)
df

,Product,Ingredient,Exp,Usage
0,Baked Potato,Potato,0.35,"Fills 5 (), while raw fills 1 ()."
1,Dried Kelp,Kelp,0.10,Fills 1 () and can be eaten faster than other ...
2,Steak,Raw Beef,0.35,"Fills 8 (), while raw fills 3 ()."
3,Cooked Porkchop,Raw Porkchop,0.35,"Fills 8 (), while raw fills 3 ()."
4,Cooked Mutton,Raw Mutton,0.35,"Fills 6 (), while raw fills 2 ()."
5,Cooked Chicken,Raw Chicken,0.35,"Fills 6 (), while raw fills 2 () and has a 30%..."
6,Cooked Rabbit,Raw Rabbit,0.35,"Fills 5 (), while raw fills 3 ()."
7,Cooked Cod,Raw Cod,0.35,"Fills 5 (), while raw fills 2 ()."
8,Cooked Salmon,Raw Salmon,0.35,"Fills 6 (), while raw fills 2 ()."


In [34]:
words = (
    pd.Series(dfstr.values.flatten())
    .str.split()
    .explode()
    .value_counts()
    .reset_index()
)

words.columns = ['Word', 'Count']

words = words[words['Word'] != 'nan']
words


,Word,Count
0,Fills,9
1,while,9
2,raw,8
3,"(),",8
4,0.35,8
5,fills,8
6,().,7
7,Raw,7
8,Cooked,6
9,2,4
